In [1]:
import awswrangler  as wr
import pandas as pd
# import pandas.io.json
import pymysql
import datetime
import json


# load sql
sql = '''
select *
from moyo_scraping.mno_scraping_result
where 1=1
'''
sql_v1 = ''' 
select *
from moyo_scraping.mno_scraping_result
where 1=1
    and version = 1 
'''
sql_v2 = '''
select *
from moyo_scraping.mno_scraping_result
where 1=1

    and version = 2
'''
df_v1= wr.athena.read_sql_query(sql=sql_v1, database="moyo_order")
df_v2= wr.athena.read_sql_query(sql=sql_v2, database="moyo_order")
df = wr.athena.read_sql_query(sql=sql, database="moyo_order")

request_result = zip(df_v1.request_id, df_v1.result)
parsed_data_v1 = []
for (request_id, result) in request_result:
    parsed_data_v1.append({'request_id': request_id, **json.loads(result)})

request_result = zip(df_v2.request_id, df_v2.result)
parsed_data_v2 = []
for (request_id, result) in request_result:
    parsed_data_v2.append({'request_id': request_id, **json.loads(result)})

request_result = zip(df.request_id, df.result)
parsed_data = []
for (request_id, result) in request_result:
    parsed_data.append({'request_id': request_id, **json.loads(result)})


# load json
# main 
df_scraping_main = pd.json_normalize (parsed_data 
                , sep = '_'
                )

bucket = 'moyo-mart'
table_name = 'scraping_main'
res = wr.s3.to_parquet(
    df=df_scraping_main,
    path=f"s3://{bucket}/{table_name}/",
    dataset=True,
    database="moyo_mart",
    table=table_name,
    mode="overwrite_partitions", 
    # partition_cols=['date_ymd']
)
#bill- v1, v2
df_scraping_bill_v1 = pd.json_normalize (parsed_data_v1 
                , sep = '_'
                , record_path= ['paymentHistory','billDetails','items'] 
                , meta = ['request_id',['paymentHistory','billedDate']]
                # ,meta_prefix="main_"
                )
df_scraping_bill_v2 = pd.json_normalize (parsed_data_v2
                , sep = '_'
                , record_path= ['billingHistory','billDetails','items'] 
                , meta = ['request_id',['paymentHistory','billedDate']]
                # ,meta_prefix="main_"
                )
df_scraping_bill = df_scraping_bill_v2.append(df_scraping_bill_v1)
bucket = 'moyo-mart'
table_name = 'scraping_bill'
res = wr.s3.to_parquet(
    df=df_scraping_bill,
    path=f"s3://{bucket}/{table_name}/",
    dataset=True,
    database="moyo_mart",
    table=table_name,
    mode="overwrite_partitions", 
    # partition_cols=['date_ymd']
)
#discount 
df_scraping_discount_v1 = pd.json_normalize (parsed_data_v1
                , sep = '_'
                , record_path= ['paymentHistory','discountDetails','items'] 
                , meta = ['request_id',['paymentHistory','paidDate']]
                # ,meta_prefix="main_"
                )
df_scraping_discount_v2 = pd.json_normalize (parsed_data_v2
                , sep = '_'
                , record_path= ['billingHistory','discountDetails','items'] 
                , meta = ['request_id',['billingHistory','billedDate']]
                # ,meta_prefix="main_"
                )
df_scraping_discount= df_scraping_discount_v2.append(df_scraping_discount_v1)

bucket = 'moyo-mart'
table_name = 'scraping_discount'
res = wr.s3.to_parquet(
    df=df_scraping_discount,
    path=f"s3://{bucket}/{table_name}/",
    dataset=True,
    database="moyo_mart",
    table=table_name,
    mode="overwrite_partitions", 
    # partition_cols=['date_ymd']
)

#usage 
df_scraping_usage = pd.json_normalize (parsed_data 
                , sep = '_'
                , record_path= ['mobileUsageHistory'] 
                , meta = ['request_id']
                # ,meta_prefix="main_"
                )

bucket = 'moyo-mart'
table_name = 'scraping_usage'
res = wr.s3.to_parquet(
    df=df_scraping_usage,
    path=f"s3://{bucket}/{table_name}/",
    dataset=True,
    database="moyo_mart",
    table=table_name,
    mode="overwrite_partitions", 
    # partition_cols=['date_ymd']
)

#family
df_scraping_family = pd.json_normalize (parsed_data 
                , sep = '_'
                , record_path= ['familyBundleMembers'] 
                , meta = ['request_id']
                # ,meta_prefix="main_"
                )

bucket = 'moyo-mart'
table_name = 'scraping_family'
res = wr.s3.to_parquet(
    df=df_scraping_family,
    path=f"s3://{bucket}/{table_name}/",
    dataset=True,
    database="moyo_mart",
    table=table_name,
    mode="overwrite_partitions", 
    # partition_cols=['date_ymd']
)


# https://s3.console.aws.amazon.com/s3/buckets/moyo-mart?region=ap-northeast-2&prefix=ringo_partition_test/&showversions=false


# v1.2 2023.05.23 에 작업함 
# billDetails paymentDetails 관련 변경사항 slack 참고 
## https://themoyo.slack.com/archives/C04VDRJ29B2/p1684821720087819


/var/folders/df/cs55_cks0_3_zx14p6v7h1_h0000gn/T/ipykernel_17157/3109222412.py:78: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_scraping_bill = df_scraping_bill_v2.append(df_scraping_bill_v1)
/var/folders/df/cs55_cks0_3_zx14p6v7h1_h0000gn/T/ipykernel_17157/3109222412.py:103: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_scraping_discount= df_scraping_discount_v2.append(df_scraping_discount_v1)
